<a href="https://colab.research.google.com/github/Rlackdals981010/ai_Study/blob/master/Lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np

In [4]:
fish = pd.read_csv('https://bit.ly/fish_csv_data')

In [5]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [6]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

In [7]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [9]:
#타겟 데이터 추출
fish_target=fish['Species'].to_numpy()

In [10]:
#훈련, 테스트 모델 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [11]:
#전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [32]:
#로지스틱(시그모이드) 회귀
##선형 회귀와 동일하게 선형 방정식 학습
bream_smelt_indexex = (train_target == 'Bream')|(train_target=='Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexex]
target_bream_smelt = train_target[bream_smelt_indexex]
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))
print(lr.classes_)#앞이 음성 클래스, 뒤는 양성
print(lr.predict_proba(train_bream_smelt[:5]))#앞이 음성 클래스에 대한 확률, 뒤는 양성
print(lr.coef_, lr.intercept_)
#z = -0.404*(Weight) - 0.576*(Length) - 0.663*(Diagonal) - 1.013*(Height) - 0.732*(Width) - 2.161
decisions = lr.decision_function(train_bream_smelt[:5])#양성 클래스에 대한 z값 반환
print(decisions)
#z값을 시그모이드 함수를 통해 확률로 변환하기
from scipy.special import expit
print(expit(decisions))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']
['Bream' 'Smelt']
[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]
[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]
[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]
[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


In [42]:
#로지스틱 회귀로 다중 분류 수행
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))
#값이 적절해 보임 -> 과대, 과소 아닌듯
print(lr.predict(test_scaled[:5]))
print(lr.classes_)
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))
#5개 샘플, 7개 생선
print(lr.coef_.shape, lr.intercept_.shape)
#특성이 5개이므로 coef_배열의 열 5
##행과 intercept가 7이라는 건 계수와 절편이 7개, 즉 z를 7개를 계산한다는 것
###z가 가장 큰 클래스가 예측 클래스
####소프트맥스 함수를 이용한다. (이진 분류는 시그모이드함수를 통해 0<z<1) <- 이 2개의 함수는 신경망에서도 사용함

0.9327731092436975
0.925
['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
(7, 5) (7,)


In [46]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision,decimals=2))
from scipy.special import softmax
proba = softmax(decision, axis=1) # 소프트맥스 함수, decision 배열을 하고, axis=1로 각 행, 즉 샘플에 대해 계산했음, 지정 안하면 배열 전체에 대해 계산함
print(np.round(proba,decimals=3))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
